In [2]:
def smart_load_and_filter_data(start_date, end_date, path='data/'):
    """
    Intelligent data loading and processing for specified date range with automatic missing file handling
    
    This function combines file discovery, robust loading, and data processing into a single streamlined
    pipeline. It automatically generates date ranges, checks for file existence, handles datetime conversion,
    merges environmental data, and calculates refractive index parameters.
    
    Parameters:
    -----------
    start_date : str
        Start date in YYYYMMDD format (e.g., '20250101')
    end_date : str
        End date in YYYYMMDD format (e.g., '20250310')
    path : str, optional
        Base directory path for data files (default: 'data/')
        
    Returns:
    --------
    pandas.DataFrame
        Processed dataset with the following columns:
        - laser_offset: Filtered laser offset values (-200 to 0)
        - temperature: Environmental temperature in °C
        - humidity: Relative humidity in %
        - pressure: Atmospheric pressure in hPa
        - counts_ratio: Calculated optical counts ratio
        - n_1762: Refractive index at 1762 nm wavelength
        Index: Timestamp (datetime index)
        
    Raises:
    -------
    ValueError
        If no complete date data is found in the specified range
    FileNotFoundError
        If required data files are missing for valid dates
        
    Example:
    --------
    >>> df = smart_load_and_filter_data('20250101', '20250310', path='experiment_data/')
    >>> print(f"Loaded {len(df)} records")
    """
    
    def generate_date_list(start_date, end_date):
        """Generate list of dates between start and end dates in YYYYMMDD format"""
        start = datetime.strptime(start_date, "%Y%m%d")
        end = datetime.strptime(end_date, "%Y%m%d")
        date_list = []
        
        current = start
        while current <= end:
            date_list.append(current.strftime("%Y%m%d"))
            current += timedelta(days=1)
        
        return date_list
    
    # Generate date range and build file path lists
    date_list = generate_date_list(start_date, end_date)
    offset_files, humidity_files, pressure_files, temp_files = [], [], [], []
    
    # Check file existence and build complete file lists
    for date in date_list:
        offset_path = os.path.join(path, f'offset_data_{date}.csv')
        humidity_path = os.path.join(path, f'humidity_data_{date}.csv')
        pressure_path = os.path.join(path, f'pressure_data_{date}.csv')
        temp_path = os.path.join(path, f'temperature_data_{date}.csv')
        
        # Only process dates with complete data files
        if all(os.path.exists(p) for p in [offset_path, humidity_path, pressure_path, temp_path]):
            offset_files.append(offset_path)
            humidity_files.append(humidity_path)
            pressure_files.append(pressure_path)
            temp_files.append(temp_path)
        else:
            print(f"Skipping {date}: Incomplete files")
    
    # Validate that we have at least one complete date
    if not offset_files:
        raise ValueError(f"No complete date data found in range {start_date} to {end_date}")
    
    print(f"Processing {len(offset_files)} complete days of data")
    
    # Initialize lists for data storage
    offset_dfs, humidity_dfs, pressure_dfs, temp_dfs = [], [], [], []
    
    # Load and validate all data files
    for offset_file, humidity_file, pressure_file, temp_file in zip(
        offset_files, humidity_files, pressure_files, temp_files
    ):
        try:
            # Load CSV files
            offset_df = pd.read_csv(offset_file)
            humidity_df = pd.read_csv(humidity_file)
            pressure_df = pd.read_csv(pressure_file)
            temp_df = pd.read_csv(temp_file)
            
            # Ensure time column is datetime type
            for df in [offset_df, humidity_df, pressure_df, temp_df]:
                if 'time' in df.columns and not pd.api.types.is_datetime64_any_dtype(df['time']):
                    df['time'] = pd.to_datetime(df['time'])
            
            # Store validated dataframes
            offset_dfs.append(offset_df)
            humidity_dfs.append(humidity_df)
            pressure_dfs.append(pressure_df)
            temp_dfs.append(temp_df)
                
        except Exception as e:
            print(f"Error loading {offset_file}: {str(e)}")
            continue
    
    # Check if we have valid data after loading
    if not offset_dfs:
        raise ValueError("All data files failed to load or were invalid")
    
    # Concatenate all data across dates
    offset_df = pd.concat(offset_dfs, ignore_index=True)
    humidity_df = pd.concat(humidity_dfs, ignore_index=True)
    pressure_df = pd.concat(pressure_dfs, ignore_index=True)
    temp_df = pd.concat(temp_dfs, ignore_index=True)
    
    # Filter valid laser offset range (-200 to 0)
    valid_offset = offset_df[(offset_df['offset'] >= -200) & (offset_df['offset'] <= 0)].copy()
    
    # Standardize column names
    humidity_df = humidity_df.rename(columns={'humidity_wm_bme280': 'humidity'})
    pressure_df = pressure_df.rename(columns={'pressure_wm_bme280': 'pressure'})
    temp_df = temp_df.rename(columns={'temperature_wm_bme280': 'temperature'})
    
    # Final datetime type verification
    for df in [valid_offset, humidity_df, temp_df, pressure_df]:
        if not pd.api.types.is_datetime64_any_dtype(df['time']):
            df['time'] = pd.to_datetime(df['time'])
    
    # Sort data for temporal merging
    sorted_offset = valid_offset.sort_values('time')
    sorted_humidity = humidity_df[['time', 'humidity']].sort_values('time')
    sorted_temp = temp_df[['time', 'temperature']].sort_values('time')
    sorted_pressure = pressure_df[['time', 'pressure']].sort_values('time')
    
    # Merge environmental data using nearest timestamp matching
    env_df = pd.merge_asof(sorted_offset, sorted_humidity, on='time', direction='nearest')
    env_df = pd.merge_asof(env_df.sort_values('time'), sorted_temp, on='time', direction='nearest')
    env_df = pd.merge_asof(env_df.sort_values('time'), sorted_pressure, on='time', direction='nearest')
    
    # Define refractive index calculation functions
    def offset2countsratio(offset, temperature, pressure, humidity):
        """
        Convert laser offset to optical counts ratio using Ciddor's refractive index equation
        
        Parameters:
        -----------
        offset : float
            Laser frequency offset in Hz
        temperature : float
            Temperature in °C
        pressure : float
            Pressure in hPa
        humidity : float
            Relative humidity in %
            
        Returns:
        --------
        float
            Optical counts ratio
        """
        f_1762 = 170.12643244933331e12  # 1762 nm laser frequency
        f_rb = 384.2281145e12  # Rubidium reference frequency
        n_rb = ciddor(wave=299792458/f_rb*1e9, t=temperature, p=pressure, rh=humidity)
        n_1762 = ciddor(wave=1762, t=temperature, p=pressure, rh=humidity)
        return n_rb/n_1762 * f_rb/((offset*1e-6 + 170.12643244933331)*1e12)

    def offset2n1762(ratio, temperature, pressure, humidity):
        """
        Convert counts ratio to refractive index at 1762 nm wavelength
        
        Parameters:
        -----------
        ratio : float
            Optical counts ratio from offset2countsratio
        temperature : float
            Temperature in °C
        pressure : float
            Pressure in hPa
        humidity : float
            Relative humidity in %
            
        Returns:
        --------
        float
            Refractive index at 1762 nm
        """
        f_1762 = 170.126432e12  # 1762 nm laser frequency
        f_rb = 384.2281145e12  # Rubidium reference frequency
        n_rb = ciddor(wave=299792458/f_rb*1e9, t=temperature, p=pressure*100, rh=humidity)
        return n_rb * f_rb / f_1762 / ratio
    
    # Calculate derived optical parameters
    env_df['counts_ratio'] = env_df.apply(
        lambda row: offset2countsratio(
            offset=row['offset'],
            temperature=row['temperature'],
            pressure=row['pressure'],
            humidity=row['humidity']
        ), axis=1
    )

    env_df['n_1762'] = env_df.apply(
        lambda row: offset2n1762(
            ratio=row['counts_ratio'],
            temperature=row['temperature'],
            pressure=row['pressure'],
            humidity=row['humidity']
        ), axis=1
    )

    # Final dataframe preparation
    final_df = env_df.set_index('time')
    final_df = final_df.rename(columns={'offset': 'laser_offset'})
    
    print(f"Successfully processed {len(final_df)} data points")
    print(f"Date range: {final_df.index.min()} to {final_df.index.max()}")
    
    return final_df
